In [2]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

import os
def init():
    # disable welcome message
    os.environ['BITSANDBYTES_NOWELCOME'] = '1'
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
init()

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

Generation config from [here](https://github.com/LAION-AI/Open-Assistant/blob/main/model/model_training/configs/ppo_config.yaml) seems to work

In [ ]:
# test
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('AlekseyKorshuk/vicuna-7b')

In [18]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from transformers import GenerationConfig
from rewardlm.utils import load_config

config = load_config('debug_RL')

rlmanager = RLModel(
    model_id = config['generation']['model_id'],
    reward_model_id = config['reward']['model_id'],
    optimized = True,   # use LoRA
    bs = config['PPO']['bs'],
    mini_bs = config['PPO']['mini_bs'],
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
    generation_config=GenerationConfig(
        max_new_tokens = 512,
        min_new_tokens = 5,
        pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
        temperature = 1,
        top_p = .7,
        top_k = 0,
        do_sample = True
        # diversity_penalty = .1, # should use num_beam_groups > 1
    )
)

data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
if config['data']['subset']:
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = rlmanager.generate_dataset(text = data)

stats = rlmanager.train_PPO(dataset = dataset)
print('Done')

# assuming debug if subset is active
if not config['data']['subset']:
    # save trainer (model, tokenizer & config) to the hub
    repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-RL-LoRA-test0'

    rlmanager.push_generator_to_hub(repo_id = repo_id)

Selected device: cpu


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 589824 || all params 125030400 || trainable(%): 0.47


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

loader len: 2


100%|██████████| 1/1 [00:00<00:00,  2.67it/s]
1it [00:11, 11.76s/it]

PPO trainer checkpoint saved


100%|██████████| 1/1 [00:00<00:00,  3.87it/s]
2it [00:25, 12.55s/it]


# 👨🏼‍🏫 Model fine-tune

In [1]:
import torch
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'      # silence warning
from rewardlm.data.data_utils import get_DIALOCONAN_prepro, get_dataset_CLM
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.utils import load_config


config = load_config('RedPajama-INCITE-Chat-3B-v1')
repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-FT-LoRA-test1'

generator_manager = GenerativeModel(
    config['generation']['model_id'],
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro()
if config['data']['subset']:
    print('getting subset')
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = get_dataset_CLM(
    data, 
    context_length = 512, 
    tokenizer = generator_manager.tokenizer
)

generator_manager.fine_tune(
    dataset = dataset.shuffle(), 
    optimized = True, # if torch.cuda.is_available() else False,
    lr = config['generation']['lr'],
    epochs = config['generation']['epochs'],
    initial_bs = config['generation']['initial_bs'],
    run_name = repo_id.split('/')[1],
)

# assuming debug if subset is active
if not config['data']['subset']:
    # save model to the hub
    generator_manager.push_to_hub(repo_id = repo_id)

bin /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
Accelerator selected device: cpu


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Map:   0%|          | 0/8312 [00:00<?, ? examples/s]

trainable params: 5242880 || all params 2781107200 || trainable(%): 0.19
Trainer device: mps


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Currently logged in as: danielsc4. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1560 [00:00<?, ?it/s]

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

### Load the fine tuned model from 🤗 Hub

In [6]:
from rewardlm.core.GenerativeModel import GenerativeModel

new_generator = GenerativeModel(
    model_id='DanielSc4/LaMini-GPT-1.5B-FT-LoRA-test0',
    load_from_peft=True,
)

print('Inference test: ')
inputs = new_generator.tokenizer("User: Woman should always be considere inferior?\nAssistant: ", return_tensors = 'pt')
new_generator.inference_fine_tuned(tokenized_batch=inputs, return_decoded=True)

Accelerator selected device: mps
Obtaining original model: MBZUAI/LaMini-GPT-1.5B
Inference test: 


'User: Woman should always be considere inferior?\nAssistant: \nWoman: Yes.'

In [18]:
text = 'Explain how to solve the immigration problem'
inputs = new_generator.tokenizer(
    "User: {text}\nAssistant: ".format(text = text), 
    return_tensors = 'pt'
)
output = new_generator.inference_fine_tuned(tokenized_batch=inputs, return_decoded=True)
print(output)

User: Explain how to solve the immigration probelem
Assistant: 

To solve the immigration probelem, you need to follow these steps:

1. Identify the source of the probelem.
2. Identify the purpose of the probelem.
3. Determine the appropriate action to take.

Steps to solve the immigration probelem:

1. Identify the source of the probelem.
2. Identify the purpose of the probelem.
3. Determine the appropriate action to take.


# ⚖️ ToxicityMeter

In [ ]:
from transformers import GenerationConfig
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.utils.general_utils import device_selector

model_id = 'bigscience/bloomz-3b'
reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'
batchsize = 16
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    # crashes while using batchsize > 1 on mps device if not set:
    pad_token_id = 0,
    temperature = 0.8,
    top_p = .8,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

load_dtype = 'bf16' # can be ['8-bit', 'bf16', 'fp32']
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = load_dtype,
    toxicity_model_id = reward_model_id,
    device = device_selector(),
    generation_config = generation_config,
)

In [ ]:
from rewardlm.data.data_utils import get_real_toxicity_prompts

custom_prompt = ('Statement: "{prompt}".\nResponse: ')
# custom prompts required by the original paper of RedPajama
if model_id == 'togethercomputer/RedPajama-INCITE-Chat-3B-v1':
    custom_prompt = ('<human>: "{prompt}"\n<bot>: ')

toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = get_real_toxicity_prompts()['text'].to_list(),
    custom_prompt = custom_prompt, 
    generation_config = generation_config,
    batch_size = batchsize,
)

In [ ]:
# save csv in tmp folder
fld = './result analysis/tmp'
toxicity_df.to_csv(
    fld + f'/measured_tox_instruct_{model_id.split("/")[-1]}_{load_dtype}.csv'
)